# Data Analytics for Health - Task 1.1.2: Merge Datasets

## Overview
This notebook merges the four healthcare datasets using two strategies:
1. **Option A**: Merge on `subject_id` only (patient-level)
2. **Option B**: Merge on `(subject_id, hadm_id)` pair (admission-level)

## Objectives
- Clean problematic hadm_ids (those with multiple subject_ids)
- Add subject_id to datasets that only have hadm_id
- Aggregate datasets appropriately
- Merge all datasets using both strategies

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# Set up file paths
notebook_dir = Path.cwd().resolve()
data_path = (notebook_dir / '..' / 'Data').resolve()

print("Libraries imported successfully")
print(f"Data path: {data_path}")

Libraries imported successfully
Data path: /Users/alexandermittet/Library/Mobile Documents/com~apple~CloudDocs/uni_life/UniPi DAD/data_analytics_4_health_unipi/Data


## 1. Load Datasets

In [2]:
# Load all four datasets
df1 = pd.read_csv(data_path / 'heart_diagnoses_1.csv')  # Heart Diagnoses
df2 = pd.read_csv(data_path / 'laboratory_events_codes_2.csv')  # Laboratory Events
df3 = pd.read_csv(data_path / 'microbiology_events_codes_3.csv')  # Microbiology Events
df4 = pd.read_csv(data_path / 'procedure_code_4.csv')  # Procedure Codes

print(f"Loaded Heart Diagnoses: {df1.shape[0]:,} rows × {df1.shape[1]} columns")
print(f"Loaded Laboratory Events: {df2.shape[0]:,} rows × {df2.shape[1]} columns")
print(f"Loaded Microbiology Events: {df3.shape[0]:,} rows × {df3.shape[1]} columns")
print(f"Loaded Procedure Codes: {df4.shape[0]:,} rows × {df4.shape[1]} columns")
print("\nAll datasets loaded successfully!")

Loaded Heart Diagnoses: 4,864 rows × 25 columns
Loaded Laboratory Events: 978,503 rows × 14 columns
Loaded Microbiology Events: 15,587 rows × 14 columns
Loaded Procedure Codes: 14,497 rows × 6 columns

All datasets loaded successfully!


## 2. Check for Problematic hadm_ids

Some hadm_ids map to multiple subject_ids, which violates data integrity. We'll identify and handle these.

In [3]:
# Function to clean datasets: remove hadm_ids with multiple subject_ids
def clean_df(df):
    """Remove rows where hadm_id maps to multiple subject_ids"""
    # if 'hadm_id' in df.columns and 'subject_id' in df.columns:
    #     # Count unique subject_ids per hadm_id
    #     counts = df.groupby('hadm_id')['subject_id'].nunique()
    #     # Keep only hadm_ids with exactly one subject_id
    #     valid_hadm = counts[counts == 1].index
    #     return df[df['hadm_id'].isin(valid_hadm)].copy()
    return df.copy()

# Check for problematic hadm_ids
print("="*80)
print("Checking for problematic hadm_ids (multiple subject_ids per hadm_id)")
print("="*80)

for name, df in [("Heart Diagnoses", df1), ("Microbiology Events", df3), ("Procedure Codes", df4)]:
    if 'hadm_id' in df.columns and 'subject_id' in df.columns:
        total_hadm = df['hadm_id'].nunique()
        problematic = df.groupby('hadm_id')['subject_id'].nunique()
        problematic_count = (problematic > 1).sum()
        pct = (problematic_count / total_hadm * 100) if total_hadm > 0 else 0
        
        print(f"\n{name}:")
        print(f"  Total unique hadm_ids: {total_hadm:,}")
        print(f"  Problematic hadm_ids (multiple subject_ids): {problematic_count}")
        print(f"  Percentage: {pct:.2f}%")
        
        if problematic_count > 0:
            examples = problematic[problematic > 1].head(5)
            for hadm, count in examples.items():
                subject_ids = df[df['hadm_id'] == hadm]['subject_id'].unique()
                print(f"    hadm_id {hadm}: {count} subject_ids -> {list(subject_ids)}")

# # Clean all datasets
df1_clean = clean_df(df1)
df2_clean = df2.copy()  # df2 doesn't have subject_id yet
df3_clean = clean_df(df3)
df4_clean = clean_df(df4)

Checking for problematic hadm_ids (multiple subject_ids per hadm_id)

Heart Diagnoses:
  Total unique hadm_ids: 4,761
  Problematic hadm_ids (multiple subject_ids): 101
  Percentage: 2.12%
    hadm_id 20200492: 2 subject_ids -> [19781816, 19998560]
    hadm_id 20222315: 2 subject_ids -> [19032473, 19998539]
    hadm_id 20343031: 2 subject_ids -> [17922874, 19998599]
    hadm_id 20624985: 2 subject_ids -> [12483604, 19998509]
    hadm_id 20706765: 2 subject_ids -> [12407830, 19998533]

Microbiology Events:
  Total unique hadm_ids: 2,454
  Problematic hadm_ids (multiple subject_ids): 256
  Percentage: 10.43%
    hadm_id 20007905.0: 3 subject_ids -> [13709807, 19997460, 19997485]
    hadm_id 20095782.0: 2 subject_ids -> [17443221, 19997450]
    hadm_id 20097155.0: 2 subject_ids -> [18048134, 19997631]
    hadm_id 20113266.0: 2 subject_ids -> [13933090, 19997491]
    hadm_id 20205373.0: 2 subject_ids -> [17844820, 19997544]

Procedure Codes:
  Total unique hadm_ids: 3,459
  Problematic had

## 3. Define Merge Functions

In [ ]:
def merge_option_a_subject_id(df1, df2, df3, df4):
    """
    Option A: Merge on subject_id only (patient-level aggregation)
    - Start with df1 (Heart Diagnoses) as base
    - Add subject_id to df2 (Labs) using reference from df1/df3
    - Aggregate all datasets by subject_id
    - Merge all on subject_id
    """
    # Create reference table for hadm_id -> subject_id mapping
    ref_table = pd.concat([
        df1_clean[['hadm_id', 'subject_id']].drop_duplicates(),
        df3_clean[['hadm_id', 'subject_id']].drop_duplicates()
    ]).drop_duplicates()
    
    # Add subject_id to df2 (Labs) - it only has hadm_id
    df2_with_subject = df2_clean.merge(ref_table, on='hadm_id', how='left')
    print(f"df2 (Labs) after adding subject_id: {df2_with_subject['subject_id'].notna().sum():,} / {len(df2_with_subject):,} rows have subject_id")
    
    # Aggregate df2 by subject_id (numeric columns: mean and count)
    numeric_cols_df2 = df2_with_subject.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols_df2 = [c for c in numeric_cols_df2 if c not in ['hadm_id', 'subject_id']]
    
    agg_dict_df2 = {}
    for col in numeric_cols_df2:
        agg_dict_df2[col] = ['mean', 'count']
    
    df2_agg = df2_with_subject.groupby('subject_id').agg(agg_dict_df2).reset_index()
    df2_agg.columns = ['subject_id'] + [f'{col}_{stat}' for col in numeric_cols_df2 for stat in ['mean', 'count']]
    
    # Aggregate df3 by subject_id
    numeric_cols_df3 = df3_clean.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols_df3 = [c for c in numeric_cols_df3 if c not in ['hadm_id', 'subject_id']]
    
    agg_dict_df3 = {}
    for col in numeric_cols_df3:
        agg_dict_df3[col] = ['mean', 'count']
    
    df3_agg = df3_clean.groupby('subject_id').agg(agg_dict_df3).reset_index()
    df3_agg.columns = ['subject_id'] + [f'{col}_{stat}' for col in numeric_cols_df3 for stat in ['mean', 'count']]
    
    # Aggregate df4 by subject_id
    numeric_cols_df4 = df4_clean.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols_df4 = [c for c in numeric_cols_df4 if c not in ['hadm_id', 'subject_id']]
    
    agg_dict_df4 = {}
    for col in numeric_cols_df4:
        agg_dict_df4[col] = ['mean', 'count']
    
    df4_agg = df4_clean.groupby('subject_id').agg(agg_dict_df4).reset_index()
    df4_agg.columns = ['subject_id'] + [f'{col}_{stat}' for col in numeric_cols_df4 for stat in ['mean', 'count']]
    
    # Start with df1 (keep all columns, but aggregate if multiple rows per subject_id)
    # For df1, we'll take the first row per subject_id (or could aggregate)
    df1_base = df1_clean.groupby('subject_id').first().reset_index()
    
    # Merge all datasets
    merged = df1_base.merge(df2_agg, on='subject_id', how='outer')
    merged = merged.merge(df3_agg, on='subject_id', how='outer')
    merged = merged.merge(df4_agg, on='subject_id', how='outer')
    
    return merged, df2_agg, df3_agg, df4_agg


def merge_option_b_subject_hadm_id(df1, df2, df3, df4):
    """
    Option B: Merge on (subject_id, hadm_id) pair (admission-level)
    - Start with df1 (Heart Diagnoses) as base
    - Add subject_id to df2 (Labs) using reference from df1/df3
    - Aggregate all datasets by (subject_id, hadm_id)
    - Merge all on (subject_id, hadm_id)
    """
    # Create reference table for hadm_id -> subject_id mapping
    ref_table = pd.concat([
        df1_clean[['hadm_id', 'subject_id']].drop_duplicates(),
        df3_clean[['hadm_id', 'subject_id']].drop_duplicates()
    ]).drop_duplicates()
    
    # Add subject_id to df2 (Labs)
    df2_with_subject = df2_clean.merge(ref_table, on='hadm_id', how='left')
    print(f"df2 (Labs) after adding subject_id: {df2_with_subject['subject_id'].notna().sum():,} / {len(df2_with_subject):,} rows have subject_id")
    
    # Aggregate df2 by (subject_id, hadm_id)
    numeric_cols_df2 = df2_with_subject.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols_df2 = [c for c in numeric_cols_df2 if c not in ['hadm_id', 'subject_id']]
    
    agg_dict_df2 = {}
    for col in numeric_cols_df2:
        agg_dict_df2[col] = ['mean', 'count']
    
    # Here we do the groupby
    df2_agg = df2_with_subject.groupby(['subject_id', 'hadm_id']).agg(agg_dict_df2).reset_index()
    df2_agg.columns = ['subject_id', 'hadm_id'] + [f'{col}_{stat}' for col in numeric_cols_df2 for stat in ['mean', 'count']]
    
    # Aggregate df3 by (subject_id, hadm_id)
    numeric_cols_df3 = df3_clean.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols_df3 = [c for c in numeric_cols_df3 if c not in ['hadm_id', 'subject_id']]
    
    agg_dict_df3 = {}
    for col in numeric_cols_df3:
        agg_dict_df3[col] = ['mean', 'count']
    
    df3_agg = df3_clean.groupby(['subject_id', 'hadm_id']).agg(agg_dict_df3).reset_index()
    df3_agg.columns = ['subject_id', 'hadm_id'] + [f'{col}_{stat}' for col in numeric_cols_df3 for stat in ['mean', 'count']]
    
    # Aggregate df4 by (subject_id, hadm_id)
    numeric_cols_df4 = df4_clean.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols_df4 = [c for c in numeric_cols_df4 if c not in ['hadm_id', 'subject_id']]
    
    agg_dict_df4 = {}
    for col in numeric_cols_df4:
        agg_dict_df4[col] = ['mean', 'count']
    
    df4_agg = df4_clean.groupby(['subject_id', 'hadm_id']).agg(agg_dict_df4).reset_index()
    df4_agg.columns = ['subject_id', 'hadm_id'] + [f'{col}_{stat}' for col in numeric_cols_df4 for stat in ['mean', 'count']]
    
    # Start with df1 - get unique (subject_id, hadm_id) pairs
    df1_base = df1_clean[['subject_id', 'hadm_id']].drop_duplicates()
    
    # Merge all datasets
    merged = df1_base.merge(df2_agg, on=['subject_id', 'hadm_id'], how='outer')
    merged = merged.merge(df3_agg, on=['subject_id', 'hadm_id'], how='outer')
    merged = merged.merge(df4_agg, on=['subject_id', 'hadm_id'], how='outer')
    
    return merged, df2_agg, df3_agg, df4_agg

Merge functions defined successfully!


## 4. Execute Both Merge Strategies

In [ ]:
# Option B: Merge on (subject_id, hadm_id) pair
print("\n" + "="*80)
print("OPTION B: Merging on (subject_id, hadm_id) pair")
print("="*80)

merged_option_b, df2_agg_b, df3_agg_b, df4_agg_b = merge_option_b_subject_hadm_id(df1_clean, df2_clean, df3_clean, df4_clean)

print(f"Base (df1): {df1_clean[['subject_id', 'hadm_id']].drop_duplicates().shape[0]:,} unique (subject_id, hadm_id) pairs")

# Get total unique pairs from df1 and df3
total_pairs = pd.concat([
    df1_clean[['subject_id', 'hadm_id']].drop_duplicates(),
    df3_clean[['subject_id', 'hadm_id']].drop_duplicates()
]).drop_duplicates().shape[0]
print(f"Total (hadm_id, subject_id) pairs from df1 and df3: {total_pairs:,}")

print(f"df2 aggregated: {df2_agg_b[['subject_id', 'hadm_id']].drop_duplicates().shape[0]:,} unique (subject_id, hadm_id) pairs")
print(f"df3 aggregated: {df3_agg_b[['subject_id', 'hadm_id']].drop_duplicates().shape[0]:,} unique (subject_id, hadm_id) pairs")
print(f"df4 aggregated: {df4_agg_b[['subject_id', 'hadm_id']].drop_duplicates().shape[0]:,} unique (subject_id, hadm_id) pairs")
print(f"\nFinal merged dataset: {merged_option_b[['subject_id', 'hadm_id']].drop_duplicates().shape[0]:,} unique (subject_id, hadm_id) pairs")
print(f"Shape: {merged_option_b.shape[0]:,} rows × {merged_option_b.shape[1]} columns")

Executing both merge strategies...

OPTION A: Merging on subject_id only


df2 (Labs) after adding subject_id: 1,166,572 / 1,166,572 rows have subject_id
Base (df1): 4,392 unique subjects
df2 aggregated: 4,687 unique subjects
df3 aggregated: 2,616 unique subjects
df4 aggregated: 3,229 unique subjects

Final merged dataset: 4,694 unique subjects
Shape: 4,694 rows × 35 columns

OPTION B: Merging on (subject_id, hadm_id) pair
df2 (Labs) after adding subject_id: 1,166,572 / 1,166,572 rows have subject_id
Base (df1): 4,864 unique (subject_id, hadm_id) pairs
Total (hadm_id, subject_id) pairs from df1 and df3: 5,166
df2 aggregated: 5,157 unique (subject_id, hadm_id) pairs
df3 aggregated: 2,756 unique (subject_id, hadm_id) pairs
df4 aggregated: 3,459 unique (subject_id, hadm_id) pairs

Final merged dataset: 5,166 unique (subject_id, hadm_id) pairs
Shape: 5,166 rows × 12 columns


## 5. Comparison Summary

In [9]:
print("\n" + "="*80)
print("Sample of Option B (first 5 rows):")
print("="*80)
print(merged_option_b.head())


Sample of Option B (first 5 rows):
   subject_id   hadm_id  valuenum_mean  valuenum_count  ref_range_lower_mean  \
0    10000980  26913865      50.305427           164.0             31.772393   
1    10000980  29654838      50.766780            59.0             28.163158   
2    10002013  24760295      53.814800            50.0             30.610000   
3    10002155  23822395      43.598140           387.0             33.267801   
4    10004457  28723315      62.915600            25.0             31.069565   

   ref_range_lower_count  ref_range_upper_mean  ref_range_upper_count  \
0                  163.0             58.486196                  163.0   
1                   57.0             60.649649                   57.0   
2                   50.0             52.862400                   50.0   
3                  382.0             55.895340                  382.0   
4                   23.0             71.017391                   23.0   

   dilution_value_mean  dilution_value_count

## 6. Save Merged Datasets

In [7]:
# Save merged datasets with task prefix
# Convert ID columns to integers before saving
if 'subject_id' in merged_option_a.columns:
    merged_option_a['subject_id'] = merged_option_a['subject_id'].astype('Int64')  # Nullable integer
if 'subject_id' in merged_option_b.columns:
    merged_option_b['subject_id'] = merged_option_b['subject_id'].astype('Int64')
if 'hadm_id' in merged_option_b.columns:
    merged_option_b['hadm_id'] = merged_option_b['hadm_id'].astype('Int64')

option_a_file = data_path / '1.1.2_merged_dataset_option_a_subject_id.csv'
option_b_file = data_path / '1.1.2_merged_dataset_option_b_subject_hadm_id.csv'

merged_option_a.to_csv(option_a_file, index=False)
print(f"✓ Saved Option A to: {option_a_file}")

merged_option_b.to_csv(option_b_file, index=False)
print(f"✓ Saved Option B to: {option_b_file}")



✓ Saved Option A to: /Users/alexandermittet/Library/Mobile Documents/com~apple~CloudDocs/uni_life/UniPi DAD/data_analytics_4_health_unipi/Data/1.1.2_merged_dataset_option_a_subject_id.csv
✓ Saved Option B to: /Users/alexandermittet/Library/Mobile Documents/com~apple~CloudDocs/uni_life/UniPi DAD/data_analytics_4_health_unipi/Data/1.1.2_merged_dataset_option_b_subject_hadm_id.csv
